In [168]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

### Read the raw dataframe

In [169]:
raw_df = pd.read_csv('data/master_data/john_hopkins_research.csv')
raw_df.drop('Unnamed: 0', axis= 1, inplace= True)

## make certain Last Update is datetime
raw_df['Last Update'] = pd.to_datetime(raw_df['Last Update'])

### US dataframe

In [170]:
us_df = raw_df[raw_df['Country/Region'] == 'US']

## Clean the US df

In [171]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

# thank you to @kinghelix and @trevormarburger for this idea
abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))

In [172]:
def get_admin(combined_key):
    if pd.isnull(combined_key):
        county = np.nan
    else:
        county = combined_key.split(',')[0]
    return county

In [173]:
def clean_state(state):
    if 'D.C.' in state or 'U.S.' in state:
        state = state
    elif '(From Diamond Princess)' in state:
        if ',' in state:
            state = abbrev_us_state[state.split('(')[0].split(',')[-1][1:-1]]
        else:
            state = 'Diamond Princess'
    elif ',' in state:
        state = abbrev_us_state[state.split(',')[-1].replace(' ', '')]
        
    return state

In [174]:
us_df['Admin2'] = us_df['Combined_Key'].apply(get_admin)

C:\Users\sahil\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [175]:
us_df['Province/State'] = us_df['Province/State'].apply(clean_state)

C:\Users\sahil\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [176]:
### US at the state level
us_states = us_df.groupby(['Province/State', 'Last Update']).sum()

In [177]:
np.unique(list(map(lambda x: x[0], us_states.index)))

array(['Alabama', 'Alaska', 'American Samoa', 'Arizona', 'Arkansas',
       'California', 'Chicago', 'Colorado', 'Connecticut', 'Delaware',
       'Diamond Princess', 'District of Columbia', 'Florida', 'Georgia',
       'Grand Princess', 'Grand Princess Cruise Ship', 'Guam', 'Hawaii',
       'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky',
       'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan',
       'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska',
       'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Northern Mariana Islands',
       'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Puerto Rico',
       'Recovered', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'US', 'United States Virgin Islands', 'Utah',
       'Vermont', 'Virgin Islands', 'Virgin Islands, U.S.', 'Virginia',
       'Washington', 'Washington, D.C.', 'West Virginia', 'Wisconsin',
      

In [178]:
us_states = us_states.reset_index()

In [179]:
us_states = us_states[['Province/State', 'Last Update', 'Confirmed', 'Deaths', 'Recovered', 'Active']]

### Initialize US transformed dataframe

In [180]:
us_transformed = us_states[['Province/State']].copy()

In [181]:
us_transformed.rename(columns={'Province/State': 'State'}, inplace=True)

In [182]:
us_transformed['Date'] = us_states['Last Update'].dt.date

In [183]:
us_transformed['ConfirmedToDate'] = us_states['Confirmed']

In [184]:
us_transformed['Active'] = us_states['Active']

### New Cases

In [185]:
us_transformed['NewConfirmed'] = us_transformed.groupby('State').diff()['ConfirmedToDate']

In [186]:
us_transformed['NewConfirmed'] = np.where(np.isnan(us_transformed['NewConfirmed']), \
                                         us_transformed['ConfirmedToDate'], \
                                         us_transformed['NewConfirmed'])

In [187]:
us_transformed['PrevNewConfirmed'] = us_transformed.groupby('State').shift()['NewConfirmed'].fillna(0)

In [193]:
def get_rate(col1, col2):
    rate = []
    for state in us_transformed['State'].unique():
        tmp = us_transformed[us_transformed['State'] == state]
        rate.extend(tmp[col1]/tmp[col2].shift())
    return np.array(rate)

In [202]:
us_transformed['GrowthRate'] = get_rate('NewConfirmed', 'ConfirmedToDate')

In [203]:
us_transformed['GrowthRate'] = us_transformed['GrowthRate'].fillna(1)

In [192]:
## new confirmed two day average
us_transformed['NewConfirmed2DayAvg'] = (us_transformed['NewConfirmed'] + us_transformed['PrevNewConfirmed'])/2

In [204]:
### growth rate two day average
us_transformed['GrowthRate2DayAvg'] = get_rate('NewConfirmed2DayAvg', 'ConfirmedToDate')

In [205]:
us_transformed['GrowthRate2DayAvg'] = us_transformed['GrowthRate2DayAvg'].fillna(1) 

In [207]:
## growth rate change
us_transformed['GrowthRateChange'] = us_transformed.groupby('State').diff()['GrowthRate2DayAvg']